In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%reload_ext rpy2.ipython

In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from kaggle.house_prices import helpers
from kaggle.house_prices import missing
from kaggle.house_prices import outliers
from kaggle.house_prices import transform_numeric as TN

combined_dataset = helpers.load_data()
combined_dataset = missing.fix_all(combined_dataset)
combined_dataset = outliers.remove_outliers(combined_dataset)
combined_dataset = combined_dataset.drop(columns=['Id'])

combinedDatasetNumeric = combined_dataset.select_dtypes(include=[np.number])
combinedDatasetNumeric.shape

(2917, 35)

In [ ]:
Trans = {
    'log':  lambda x: np.log(x + 1),
    'sqrt': lambda x: np.sqrt(x),
    'inv3': lambda x: x**(1/3),
    'inv4': lambda x: x**(1/4)
}

tran_config = TN.get_transformation_config(
    df=(
        combined_dataset
        .select_dtypes(include=[np.number])
        .drop(columns=['SalePrice'])
    ),
    trans=Trans
)